# 1. Introduction

In the last mission, we learned a lot about about how decorators work. In this mission, we'll continue learning more about `decorators as we work with real world decorators and learn how to write decorators that take arguments.`

**`Memoizing is the process of storing the results of a function so that the next time the function is called with the same arguments, you can just look up the answer.`**

`def memoize(func):
    """Store the results of the decorated function for fast lookup
    """

    # Store results in a dict that maps arguments to results
    cache = {}

    def wrapper(*args, **kwargs):
        # If these arguments haven't been seen before, call func() and store the result.
        if (args, kwargs) not in cache:        
            cache[(args, kwargs)] = func(*args, **kwargs)          
        return cache[(args, kwargs)]

    return wrapper`

When the new function gets called, we check to see whether we've ever seen these arguments before. If we haven't, we send them to the decorated function, and store the result in the cache dictionary.

## TODO:
You're working on a project, and you are curious about how many times each of the functions in it gets called. So you decide to write a decorator that adds a counter to each function that you decorate. You could use this information in the future to determine whether there are sections of code that you could remove because they are no longer being used. To uncomment the lines in the code editor so you can modify them, select all of these lines and press ctrl + / (PC) or ⌘ + / (Mac).

* Call the function being decorated and return the result.
* Return the new decorated function.
* Decorate foo() with the counter() decorator.

In [1]:
def counter(func):
    def wrapper(*args, **kwargs):
        wrapper.count += 1
        # Call the function being decorated and return the result
        return func(*args, **kwargs)
    wrapper.count = 0
    # Return the new decorated function
    return wrapper

# Decorate foo() with the counter() decorator
@counter
def foo():
    print('calling foo()')

foo()
foo()

print('foo() was called {} times.'.format(foo.count))

calling foo()
calling foo()
foo() was called 2 times.


# 2. Real World Decorators

The timer() decorator runs the decorated function and then prints how long it took for the function to run. It's good to add some version of this to all of your projects because it is a pretty easy way to figure out where your computational bottlenecks are.

Like most decorators, we'll start off by defining a wrapper() function. This is the function that the decorator will return. wrapper() takes any number of positional and keyword arguments, so that it can be used to decorate any function.

## TODO:
You are debugging a package that you've been working on with your friends. Something weird is happening with the data being returned from one of your functions, but you're not even sure which function is causing the trouble. You know that sometimes bugs can sneak into your code when you are expecting a function to return one thing and it returns something different. For instance, if you expect a function to return a numpy array, but it returns a list, you can get unexpected behavior. In order to ensure this is not the cause of the trouble, you decide to write a decorator, print_return_type(), that will print out the type of the variable that gets returned from every call of any function it is decorating.

* Write a decorator, print_return_type(), that will print out the type of the variable that gets returned from every call of any function it is decorating.
  * Create a decorator named print_return_type() that takes one argument calledfunc:
    * Create a nested function, wrapper(), that will become the new decorated function. Use *args and **kwargs to allow wrapper() to take any number of positional and keyword arguments. Inside the body of wrapper:
      * Call func(), the function being decorated, and assign the result to a variable named result.
      * Print '{}() returned type {}'.format(func.__name__, type(result)) to print the type of variable.
      * Return result.
  * Return the new decorated function wrapper().
* Decorate foo() with the print_return_type() decorator.
* Use foo() to return the types of 42, [1, 2, 3], and {'a': 42}. Print the results.

In [2]:
def foo(value):
    return value
def print_return_type(func):
    # Define wrapper(), the decorated function
    def wrapper(*args, **kwargs):
        # Call the function being decorated
        result = func(*args, **kwargs)
        print('{}() returned type {}'.format(
        func.__name__, type(result)
        ))
        return result
    # Return the decorated function
    return wrapper
  
@print_return_type
def foo(value):
     return value

print(foo(42))
print(foo([1, 2, 3]))
print(foo({'a': 42}))

foo() returned type <class 'int'>
42
foo() returned type <class 'list'>
[1, 2, 3]
foo() returned type <class 'dict'>
{'a': 42}


# 3. Preserving Metadata When Decorating Functions

`One of the problems with decorators is that they obscure the decorated function's metadata.`

## TODO:
* Decorate sleep_n_seconds() with the timer() decorator.
* Use the __doc__ attribute to print the docstring for sleep_n_seconds().
* Use the __name__ attribute to print the name of sleep_n_seconds().

In [3]:
def timer(func):
    """A decorator that prints how long a function took to run."""  
    def wrapper(*args, **kwargs):
        t_start = time.time()
    
        result = func(*args, **kwargs)
    
        t_total = time.time() - t_start
        print('{} took {}s'.format(func.__name__, t_total))
    
        return result
    return wrapper

def sleep_n_seconds(n=10):
    """Pause processing for n seconds.
  
    Args:
        n (int): The number of seconds to pause for.
    """
    time.sleep(n)
@timer
def sleep_n_seconds(n=10):
    """Pause processing for n seconds.
  
    Args:
        n (int): The number of seconds to pause for.
    """
    time.sleep(n)
print(sleep_n_seconds.__doc__)
print(sleep_n_seconds.__name__)

None
wrapper


# 4. Preserving Metadata When Decorating Functions Continued

In the last exercise, we saw that when we try to print the docstring for sleep_n_seconds(), we get nothing back. Even stranger, if we try to look up the function's name, Python tells us that sleep_n_seconds()'s name is "wrapper".

Remember that when we write decorators we almost always define a nested function to return. Because the decorator overwrites the sleep_n_seconds() function, when we ask for sleep_n_seconds()'s docstring or name, we are actually referencing the nested function that was returned by the decorator. In this case, the nested function was called wrapper() and it didn't have a docstring.

Fortunately, Python provides us with an easy way to fix this. The wraps() function from the functools module is a decorator that we use when defining a decorator. If we use it to decorate the wrapper function that our decorator returns, it will modify wrapper()'s metadata to look like the function we are decorating.

`from functools import wraps`

## TODO:
Your friend has come to you with a problem. They've written some decorators and added them to the functions in the open source library they've been working on. However, they were running some tests and discovered that all of the docstrings have mysteriously disappeared from their decorated functions. Show your friend how to preserve docstrings and other metadata when writing decorators. 

* Import a function that will allow you to preserve the metadata for the decorated version of a function.
* Decorate wrapper() so that the metadata from func() is preserved in the new decorated function.
* Decorate print_sum() with the add_hello() decorator.
* Call print_sum() with the arguments 10 and 20.
* Print the docstring for print_sum().

In [4]:
from functools import wraps
def add_hello(func):
    # Decorate wrapper() so that it keeps func()'s metadata
    @wraps(func)
    def wrapper(*args, **kwargs):
        """Print 'hello' and then call the decorated function."""
        print('Hello')
        return func(*args, **kwargs)
    return wrapper

@add_hello
def print_sum(a, b):
    """Adds two numbers and prints the sum"""
    print(a + b)

print_sum(10, 20)
print(print_sum.__doc__)

Hello
30
Adds two numbers and prints the sum


# 5. Adding Arguments to Decorators

`Sometimes it would be nice to add arguments to our decorators. To do that, we need another level of nesting in our decorators.`

In [5]:
#Let's consider this silly run_three_times() decorator. If you use it to decorate a function, it will run that function three times.
def run_three_times(func):
    def wrapper(*args, **kwargs):
        for i in range(3):
            func(*args, **kwargs)
    return wrapper


#If we use it to decorate the print_sum() function and then run print_sum(3,5), it will print 8 three times.

@run_three_times
def print_sum(a, b):
    print(a + b)

print_sum(3, 5)

8
8
8


Let's think about what we would need to change if we wanted to write a run_n_times() decorator. We want to pass n in as an argument, instead of hard-coding in the decorator.

In [6]:
'''def run_n_times(func):
    def wrapper(*args, **kwargs):
        # How do we pass "n" into this function?
        for i in range(???):
            func(*args, **kwargs)
    return wrapper'''

'def run_n_times(func):\n    def wrapper(*args, **kwargs):\n        # How do we pass "n" into this function?\n        for i in range(???):\n            func(*args, **kwargs)\n    return wrapper'

If we had some way to pass n into the decorator, we could decorate print_sum() so that it gets run three times and decorate the print_hello() function to run five times.

In [7]:
@run_n_times(3)
def print_sum(a, b):
    print(a + b)

NameError: name 'run_n_times' is not defined

`But a decorator is only supposed to take one argument - the function it is decorating. Also, when we use decorator syntax, we're not supposed to use the parentheses. So how can this be done?`

To make run_n_times() work, we have to turn it into a function that returns a decorator, rather than a function that is a decorator.

# 6. Adding Arguments to Decorators Continued

In [8]:
def run_n_times(n):
    """Define and return a decorator"""
    def decorator(func):
        def wrapper(*args, **kwargs):
            for i in range(n):
                func(*args, **kwargs)
        return wrapper
    return decorator

def print_sum(a, b):
    print(a + b)

def print_hello():
    print('Hello!')
#SOLUTION
@run_n_times(3)
def print_sum(a, b):
    print(a + b)

print_sum(3, 5)

@run_n_times(5)
def print_hello():
    print('Hello!')

print_hello()

8
8
8
Hello!
Hello!
Hello!
Hello!
Hello!


# 7. Real World Decorators with Arguments

Let's finish up by looking at an example of a real world decorator that takes an argument to get a better sense of how they work.

It would be nice if we could add some kind of timeout() decorator to those functions that will raise an error if the function runs for longer than expected.

To create the timeout() decorator, we are going to use some functions from Python's signal module. These functions have nothing to do with decorators, but understanding them will help us understand the timeout() decorator.

The signal() function tells Python, "When you see the signal whose number is signalnum, call the handler function." In this case, we tell Python to call raise_timeout() whenever it sees the alarm signal.

`def timeout_in_5s(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        # Set an alarm for 5 seconds
        signal.alarm(5)
        try:
            # Call the decorated func
            return func(*args, **kwargs)
        finally:
            # Cancel alarm
            signal.alarm(0)
    return wrapper`

In [9]:
import signal
def timeout(n_seconds):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            # Set an alarm for n seconds
            signal.alarm(n_seconds)
            try:
               # Call the decorated func
               return func(*args, **kwargs)
            finally:
               # Cancel alarm
               signal.alarm(0)
        return wrapper
    return decorator

@timeout(20)
def bar():
    time.sleep(10)
    print('bar!')
bar()

AttributeError: module 'signal' has no attribute 'alarm'

# 8. Real World Decorators with Arguments Continued

In the above exercise, we created a more useful version of the timeout() decorator. This decorator takes an argument that allows us to set a timeout that is appropriate for each function.

Notice that wrapper() returns the result of calling func(), decorator() returns wrapper, and timeout() returns decorator.

timeout() is now a function that returns a decorator. You can think of it as a decorator factory. When you call timeout(), it cranks out a brand new decorator that times out in 5 seconds, or 20 seconds, or whatever value we pass as n_seconds.

So when we called bar(), which has a 20-second timeout, it printed its message in 10 seconds, so the alarm got canceled.

In [10]:
def tag(*tags):
    # Define a new decorator, named decorator(), to return
    def decorator(func):
        # Ensure the decorated function keeps its metadata
        @wraps(func)
        def wrapper(*args, **kwargs):
            # Call the function being decorated and return the result
            return func(*args, **kwargs)
        wrapper.tags = tags
        return wrapper
    # Return the new decorator
    return decorator

@tag('test', 'this is a tag')
def foo():
    pass

print(foo.tags)

('test', 'this is a tag')


In this mission, you learned more about decorators,` including how to use functools.wraps() to make sure your decorated functions maintain their metadata and how to write decorators that take arguments`.